In [35]:
#Use 3.12.4
import openai
import os
import copy
import time
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.docstore.document import Document

key_path = '/Users/jaesolshin/key/openai_key.txt'
os.environ["OPENAI_API_KEY"] = open(key_path, 'r', encoding='utf-8').read()
openai.api_key = os.environ["OPENAI_API_KEY"]

In [36]:
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

import fitz  # PyMuPDF
from pykospacing import Spacing

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

year=2019
pdf_path = f'/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/{year}년도 연차보고서.pdf'
extracted_text = extract_text_from_pdf(pdf_path)

spacing = Spacing()
corrected_text = spacing(extracted_text)

In [37]:
corrected_documents = Document(page_content=corrected_text)

In [38]:
from langchain.text_splitter import SpacyTextSplitter
from langchain.docstore.document import Document

corrected_documents = [Document(page_content=corrected_text)]

text_splitter = SpacyTextSplitter(
    chunk_size=2000, #분할할 크기를 설정
    chunk_overlap=10,
    pipeline='ko_core_news_sm' #분할에 사용할 언어모델을 설정
)

splitted_documents = text_splitter.split_documents(corrected_documents) #문서를 분할

print(f'분할 전 문서 개수: {len(corrected_documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')

Created a chunk of size 5369, which is longer than the specified 2000


분할 전 문서 개수: 1
분할 후 문서 개수: 89


In [39]:
for doc in splitted_documents:
    print(doc)

page_content='ISSN 1975-4922 2019 연차보고서 2020.\n\n3 2019 연차보고서 2020.\n\n3 「한국은행법 」 제102조(연차보고서의 공표) ① \x07 한국은행은 매 회계연도가 지난 후 3개월 이내에 해 당 회계연도 중의 한국은행의 업무상태와 통화 및 정부의 외환에 관한 정책을 개략적으로 기술하고 금융경제 상태를 분석한 연차보고서를 정부에 제출하고, 이를  공표하여야 한다.\n\n② 제1항에 따른 연차보고서는 금융통화위원회의 의결을 거쳐야 한다.\n\n한국은행의 설립 목적 · 효율적인 통화신용정책의 수립과 집행을 통하여 물가 안정을 도 모함으로써 국민경제의 건전한 발전에 이 바지함 · 통화신용정책을 수행함에 있어 금융안정에 유의하여야 함 한국은행은 우리나라의 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가 안정과  금융안정을 도 모하고 국민경제가 건전하게 발전할 수 있 도록 최선을 다하고 있습니다.\n\n2019년 우리 경제는 대외여건의 악화로 많은 어려움을 겪었습니다.\n\n미·중 무역 분쟁의 영향으로 세계 교역이 위축된 가운데 주요 수출 품목인 반도체의 업황 회복도 지연되었습니다.\n\n그 과정에서 금융·외환시장이 수 시로 높은 변동성을 보였으며 경제주체들의 심리도 위축되었습니다 .\n\n이 에 따라 경제성장률이 2%로 둔화되었으며, 물가 상승률은 성장세 둔화 이 외에도 유가 하락, 정부 복지정책 등의 영향이 더해지면서 0%대의 낮은 수준에 머물렀습니다.\n\n이러한 금융경제 여건 하에서 금융통화위원회는 7월과 10월 두 번에 걸 쳐 기준금리를 0.5% 포인트 인하하여 연 1.25%로 조정하였습니다.\n\n이는 경기 회복을 뒷받침하고 물가 둔화  압력을 완화하기 위해 통화정책의 완 화 정도를 확대할 필요가 있다고 판단하였기 때문입니다.\n\n한 편 저물가 상 황이 상당 기간 지속될 수 있다는 우려가 제기됨에 따라 물가 상황 전반에 대한 경제주체들의 이 해를 높이기 위해 정책 커뮤니케이션 강화에도 많은 총재 서문 노

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 모델 정의
chat = ChatOpenAI(
    model='gpt-4o-mini-2024-07-18',
)


# 2단계 프롬프트(표, 목차 제거)
examples2 = [
    {
        'input':"Ⅱ- 1. 주요 이슈 44 Ⅱ- 2.",
        'output':"\n"
    },
    {
        'input':"표 Ⅱ- 13. 외화자산 구성 내역 (기말 기준) (%, %p) 2021 2022(A) 2023(B) 증감(B-A) ",
        'output':"\n"
    },
    {
        'input':"127) 한국은행의 업무 관련 지식, 정책 수행 경험...",
        'output':"\n"
    },
    {
        'input':"주요 품목별 수출 (%)\n 2022 2023\n 증감률 비중 증감률 비중\n IT제품 2.5 <34.1> -19.9 <29.5> ...",
        'output':"\n"
    },
]

prompt = PromptTemplate(
    input_variables=['input', 'output'],
    template='입력: {input}\n출력: {output}'
)


few_shot_prompt = FewShotPromptTemplate(
    examples=examples2,
    example_prompt=prompt,
    prefix="예시와 같이 표나 차례, 주석 형식의 불필요한 텍스트를 제거하세요. 1) 한국어 문법에 맞는 문장이 아닌 기호나 숫자, 단어 등이 나열된 목차 형식의 텍스트 제거한다. 2) 기호나 숫자, 단어 등이 나열된 표 형식의 데이터 제거한다. 3) 주석은 '127)'과 같이 숫자로 시작하는 문장은 주석이므로 제거한다. 4)  문체는 '입니다'가 아닌 '이다'체로 통일한다.",
    suffix="입력: {input_string}\n출력:",
    input_variables=['input_string'],
)

rectified_documents_step2 =[]

def rectify2(phrase):
    formatted_prompt = few_shot_prompt.format(
        input_string=phrase
    )
    result = chat(
        [
            SystemMessage(content='당신은 신문사에서 전문적인 교열을 담당하는 편집자입니다. 요구사항에 맞게 주어진 텍스트를 수정하세요.'),
            HumanMessage(content=formatted_prompt)
        ]
    )
    
    return result.content


for idx, doc in enumerate(splitted_documents):
    print(f'processing {idx+1}')
    rect = rectify2(doc.page_content)
    
    # '입력:'이나 '출력:'이 포함되어 있는지 확인
    if '입력:' not in rect and '출력:' not in rect:
        rectified_doc = Document(page_content=rect, metadata=doc.metadata)
        rectified_documents_step2.append(rectified_doc)
    else:
        print(f'Skipping document {idx+1} due to "입력:" or "출력:" being present.')
    
    time.sleep(1)


processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10
processing 11
processing 12
processing 13
processing 14
processing 15
processing 16
Skipping document 16 due to "입력:" or "출력:" being present.
processing 17
Skipping document 17 due to "입력:" or "출력:" being present.
processing 18
processing 19
processing 20
processing 21
processing 22
processing 23
processing 24
processing 25
processing 26
processing 27
Skipping document 27 due to "입력:" or "출력:" being present.
processing 28
Skipping document 28 due to "입력:" or "출력:" being present.
processing 29
processing 30
processing 31
processing 32
Skipping document 32 due to "입력:" or "출력:" being present.
processing 33
processing 34
Skipping document 34 due to "입력:" or "출력:" being present.
processing 35
processing 36
processing 37
processing 38
processing 39
Skipping document 39 due to "입력:" or "출력:" being present.
processing 40
processing 41
processing 42
processing 43
p

In [41]:
for doc in rectified_documents_step2:
    print(doc.page_content)

한국은행은 우리나라의 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가 안정과 금융 안정을 도모하고 국민경제가 건전하게 발전할 수 있도록 최선을 다하고 있다.

2019년 우리 경제는 대외 여건의 악화로 많은 어려움을 겪었다. 미·중 무역 분쟁의 영향으로 세계 교역이 위축된 가운데 주요 수출 품목인 반도체의 업황 회복도 지연되었다. 그 과정에서 금융·외환시장이 수시로 높은 변동성을 보였으며 경제주체들의 심리도 위축되었다. 이로 인해 경제 성장률이 2%로 둔화되었으며, 물가 상승률은 성장세 둔화 이외에도 유가 하락, 정부 복지정책 등의 영향이 더해지면서 0%대의 낮은 수준에 머물렀다.

이러한 금융경제 여건 하에서 금융통화위원회는 7월과 10월 두 번에 걸쳐 기준금리를 0.5% 포인트 인하하여 연 1.25%로 조정하였다. 이는 경기 회복을 뒷받침하고 물가 둔화 압력을 완화하기 위해 통화정책의 완화 정도를 확대할 필요가 있다고 판단하였기 때문이다.

한편 저물가 상황이 상당 기간 지속될 수 있다는 우려가 제기됨에 따라 물가 상황 전반에 대한 경제주체들의 이해를 높이기 위해 정책 커뮤니케이션 강화에도 많은 노력을 기울였다. 특히, 6월과 12월에 물가 안정목표 운영상황 점검 보고서를 발간하고 기자 간담회를 개최하여 물가 여건과 전망, 정책 방향 등을 소상히 설명하였다.

또한 국제결제은행의 의사결정기구인 이사회 구성원으로서 국제금융 관련 글로벌 논의에 적극 참여하였고 UAE 중앙은행과 통화스왑 계약을 재체결하는 등 중층적 외환안전망 유지에도 힘썼다. 국제금융시장의 변동성이 확대될 경우 비상대응체제를 즉각 가동하여 대응 방안을 점검하는 등 국내 금융·외환시장이 조기에 안정될 수 있도록 하였다.

아울러 우리 경제를 둘러싼 환경 변화에 능동적이고 선제적으로 대처하고자 노력하였다. 저출산·고령화 추세, 디지털 기술혁신 등에 발맞춰 관련 조사 연구를 강화하는 방향으로 조직을 개편하였다. 또한 조사연구, 지급결제 업무 관련 IT시스템을 개선하고 확충하는 등 조직의 생산성

In [42]:
txt = ""

for doc in rectified_documents_step2:
    txt += doc.page_content

#txt = txt.replace("\n", " ")  # "\n" 제거

In [43]:
print(txt)
open(f'rectified_text_{year}.txt', 'w', encoding='utf-8').write(txt)

한국은행은 우리나라의 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가 안정과 금융 안정을 도모하고 국민경제가 건전하게 발전할 수 있도록 최선을 다하고 있다.

2019년 우리 경제는 대외 여건의 악화로 많은 어려움을 겪었다. 미·중 무역 분쟁의 영향으로 세계 교역이 위축된 가운데 주요 수출 품목인 반도체의 업황 회복도 지연되었다. 그 과정에서 금융·외환시장이 수시로 높은 변동성을 보였으며 경제주체들의 심리도 위축되었다. 이로 인해 경제 성장률이 2%로 둔화되었으며, 물가 상승률은 성장세 둔화 이외에도 유가 하락, 정부 복지정책 등의 영향이 더해지면서 0%대의 낮은 수준에 머물렀다.

이러한 금융경제 여건 하에서 금융통화위원회는 7월과 10월 두 번에 걸쳐 기준금리를 0.5% 포인트 인하하여 연 1.25%로 조정하였다. 이는 경기 회복을 뒷받침하고 물가 둔화 압력을 완화하기 위해 통화정책의 완화 정도를 확대할 필요가 있다고 판단하였기 때문이다.

한편 저물가 상황이 상당 기간 지속될 수 있다는 우려가 제기됨에 따라 물가 상황 전반에 대한 경제주체들의 이해를 높이기 위해 정책 커뮤니케이션 강화에도 많은 노력을 기울였다. 특히, 6월과 12월에 물가 안정목표 운영상황 점검 보고서를 발간하고 기자 간담회를 개최하여 물가 여건과 전망, 정책 방향 등을 소상히 설명하였다.

또한 국제결제은행의 의사결정기구인 이사회 구성원으로서 국제금융 관련 글로벌 논의에 적극 참여하였고 UAE 중앙은행과 통화스왑 계약을 재체결하는 등 중층적 외환안전망 유지에도 힘썼다. 국제금융시장의 변동성이 확대될 경우 비상대응체제를 즉각 가동하여 대응 방안을 점검하는 등 국내 금융·외환시장이 조기에 안정될 수 있도록 하였다.

아울러 우리 경제를 둘러싼 환경 변화에 능동적이고 선제적으로 대처하고자 노력하였다. 저출산·고령화 추세, 디지털 기술혁신 등에 발맞춰 관련 조사 연구를 강화하는 방향으로 조직을 개편하였다. 또한 조사연구, 지급결제 업무 관련 IT시스템을 개선하고 확충하는 등 조직의 생산성

88590

In [44]:
import subprocess
subprocess.run(['say', '-v', 'Kyoko', 'プロセスが完了しました。'])

Python(8840) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


CompletedProcess(args=['say', '-v', 'Kyoko', 'プロセスが完了しました。'], returncode=0)